### **Datos COES hasta hoy** 

In [1]:

from selenium import webdriver

import datetime

import requests
from bs4 import BeautifulSoup

import pandas as pd

### **COES**

In [36]:
from datetime import  datetime

urlRoot = "https://www.coes.org.pe"
siteRoot = "/Portal/"
controlador = siteRoot + 'portalinformacion/'
url = controlador + "exportardemanda"

fechaInicial  = "06/03/2024"
fechaFinal = datetime.now().strftime('%d/%m/%Y')


fechaInicialStr = fechaInicial
fechaFinalStr = fechaFinal

webCOES = urlRoot + url

data = {
    'fechaInicial': fechaInicialStr,
    'fechaFinal': fechaFinalStr
}

# Realizar la solicitud GET a la página web

response = requests.post(webCOES, data=data)

# Comprobar si la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Analizar el HTML de la página web

    url = controlador + "descargardemanda"
    webCOES = urlRoot + url
    response = requests.get(webCOES, data=data)

    with open('DemandaCOES2.xlsx', 'wb') as f:
            f.write(response.content)

else:
    print('error')

### **COBRE**

In [2]:
from datetime import  timedelta

fechaActual = datetime.datetime.now()
fechaInicial  = fechaActual - timedelta(days=365*2)

fechaActualStr = fechaActual.strftime('%Y-%m-%d')
fechaInicialStr = fechaInicial.strftime('%Y-%m-%d')

webCOBRE = 'https://estadisticas.bcrp.gob.pe/estadisticas/series/diarias/resultados/PD04701XD/html/' 
webCOBRE = webCOBRE + fechaInicialStr + '/' + fechaActualStr

# Realizar la solicitud GET a la página web
response = requests.get(webCOBRE)

resultado =[]

# Comprobar si la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Analizar el HTML de la página web
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Encontrar todos los elementos <td> con la clase 'periodo'
    periodos = soup.find_all('td', class_='periodo')
    
    # Encontrar todos los elementos <td> con la clase 'dato'
    datos = soup.find_all('td', class_='dato')

    periodos = [periodo.text.strip() for periodo in periodos]
    datos = [dato.text.strip() for dato in datos]
    
    
else:
    print("La solicitud GET no fue exitosa. Código de estado:", response.status_code)

# Crear el DataFrame
df = pd.DataFrame({'fecha': periodos, 'valor': datos})

# viene con un error en el nombre de septiembre, pues viene Set en vez de Sep
df['fecha'] = df['fecha'].str.replace('Set', 'Sep')

import locale
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')  # Configurar el locale en español

df['fechaYYYYMMDD'] = pd.to_datetime(df['fecha'], format='%d%b%y')

In [3]:

# Crear un rango de fechas desde el 2021-01-01 hasta el 2024-12-31

rangoFechas = pd.date_range(start=fechaInicial, end=fechaActual, freq='D')
rangoFechas = rangoFechas.strftime('%Y-%m-%d')

# Crear un DataFrame con las fechas del rango
dfFechasCompletas = pd.DataFrame({'fechaYYYYMMDD': rangoFechas})
dfFechasCompletas['fechaYYYYMMDD'] = pd.to_datetime(dfFechasCompletas['fechaYYYYMMDD'], format='%Y-%m-%d')

# Hacer un merge con el DataFrame original usando left join para mantener las fechas del rango
dfResultado = pd.merge(dfFechasCompletas, df, on='fechaYYYYMMDD', how='left')

# Llenar los valores faltantes con el último valor conocido (forward fill)
dfResultado['valor'].ffill(inplace=True)
# Ver el resultado
df = dfResultado.copy()

df = df[['fechaYYYYMMDD','valor']]

df.rename(columns={'fechaYYYYMMDD': 'fecha'}, inplace=True)

df['fecha'] = df['fecha'].dt.strftime('%Y-%m-%d')

df.to_excel('datosCobre.xlsx')


/var/folders/tx/1tx9w9fj6v78n5f12zkwg_vc0000gn/T/ipykernel_19133/3324058987.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dfResultado['valor'].ffill(inplace=True)


In [4]:
# ahora vamos a mover el archivo al directorio de cartones_america
import os
import datetime

# Ruta del directorio en el que deseas buscar el archivo
directorioOrigen = "/Users/jaimereinoso/Downloads"
directorioDestino = "."

# Obtiene la fecha actual
hoy = datetime.date.today()

# Nombre del archivo que deseas buscar
nombreArchivo = "DemandaCOES.xlsx"


print(hoy)

# Ruta completa del archivo que estás buscando
rutaOrigen = os.path.join(directorioOrigen, nombreArchivo)
rutaDestino = os.path.join(directorioDestino, nombreArchivo)

# Verifica si el archivo existe en el directorio
if os.path.exists(rutaOrigen):
    # Obtiene la fecha de modificación del archivo
    fecha_modificacion = datetime.date.fromtimestamp(os.path.getmtime(rutaOrigen))

    # Compara la fecha de modificación con la fecha actual
    if fecha_modificacion == hoy:
        print(f"El archivo '{nombreArchivo}' existe en el directorio y es del día de hoy.")
    else:
        print(f"El archivo '{nombreArchivo}' existe en el directorio, pero no es del día de hoy.")
else:
    print(f"El archivo '{nombreArchivo}' no existe en el directorio.")

import os
import shutil

# Mover el archivo a la carpeta de destino y sobrescribir si es necesario
try:
    os.replace(rutaOrigen, rutaDestino)
    print("Archivo movido exitosamente.")
except OSError as e:
    print(f"No se pudo mover el archivo: {e}")


2024-03-07
El archivo 'DemandaCOES.xlsx' existe en el directorio y es del día de hoy.
Archivo movido exitosamente.
